In [1]:
import polars as pl
import pandas as pd

# 讀取 parquet
train_filled = pl.read_parquet("data/train_filled.parquet")

# 讀取top50 features
n_top = 20
save_dir = "model_output/selected_features_xgb/one_model"
df = pd.read_csv("model_output/all_features_xgb/model_importance/common_features_with_ranks.csv")
top50_features = df["feature"].head(n_top).tolist()

# 你要保留的欄位 (Top50 + target + group id)
cols_to_keep = top50_features + ["selected", "ranker_id"]

# 只保留這些欄位
train_filled = train_filled.select(cols_to_keep)

print(f"✅ DataFrame 現在只有 {len(train_filled.columns)} 欄位: {train_filled.columns}")


✅ DataFrame 現在只有 22 欄位: ['total_num_transfers_rank', 'total_is_min_transfers', 'legs0_num_transfers_rank', 'price_from_median_zscore', 'price_percentile', 'legs0_segments0_flightNumber', 'price_per_duration', 'price_per_duration_rank', 'companyID', 'has_access_tp', 'pricingInfo_isAccessTP', 'days_before_departure', 'legs0_mean_cabin', 'price_minus_fee_rank', 'price_per_fee', 'price_per_fee_rank', 'both_legs_carrier_all_same', 'legs0_arrivalAt_hour', 'legs0_departureAt_hour', 'price_per_tax', 'selected', 'ranker_id']


In [2]:
import numpy as np
import polars as pl

# 排除列
exclude_cols = ['Id', 'ranker_id', 'selected']
feature_cols = [col for col in train_filled.columns if col not in exclude_cols]
print(f"Using {len(feature_cols)} features")

# 我們先把全部需要的欄位一次select
df_all = train_filled.select(['ranker_id'] + ['selected'] + feature_cols)

# ranker_id去重
unique_rankers = df_all.select("ranker_id").unique().to_series().to_list()

# 打亂
np.random.seed(42)
np.random.shuffle(unique_rankers)

# 切8:2
n_train = int(0.8 * len(unique_rankers))
train_rankers = set(unique_rankers[:n_train])

# 標記is_train
df_all = df_all.with_columns(
    pl.col("ranker_id").is_in(train_rankers).alias("is_train")
)

# 分割
df_train = df_all.filter(pl.col("is_train"))
df_val = df_all.filter(~pl.col("is_train"))

# 切X/y
X_train_np = df_train.select(feature_cols).to_numpy()
y_train_np = df_train["selected"].to_numpy()
X_val_np = df_val.select(feature_cols).to_numpy()
y_val_np = df_val["selected"].to_numpy()

print("✅ 完成資料切分")
print("Train size:", X_train_np.shape, " Val size:", X_val_np.shape)


Using 20 features
✅ 完成資料切分
Train size: (14445656, 20)  Val size: (3699716, 20)


In [3]:
from lightgbm import early_stopping, log_evaluation, LGBMClassifier

clf = LGBMClassifier(
    objective="binary",
    class_weight="balanced",
    learning_rate=0.05,
    num_leaves=128,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=10.0,
    random_state=42,
    n_estimators=500,
    n_jobs=-1
)

clf.fit(
    X_train_np,
    y_train_np,
    eval_set=[(X_val_np, y_val_np)],
    eval_metric=["binary_logloss", "auc"],
    callbacks=[
        early_stopping(stopping_rounds=50, verbose=True),
        log_evaluation(20)
    ],
    feature_name=feature_cols  # ✅ 在這裡明確告訴LightGBM特徵名稱
)



# 預測機率
y_val_pred_proba = clf.predict_proba(X_val_np)[:,1]
print("前10筆機率:", y_val_pred_proba[:10])

from sklearn.metrics import classification_report, confusion_matrix

y_val_pred_label = clf.predict(X_val_np)

print(classification_report(y_val_np, y_val_pred_label))
print(confusion_matrix(y_val_np, y_val_pred_label))




[LightGBM] [Info] Number of positive: 84431, number of negative: 14361225
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.613907 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2743
[LightGBM] [Info] Number of data points in the train set: 14445656, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Training until validation scores don't improve for 50 rounds
[20]	valid_0's binary_logloss: 0.432191	valid_0's auc: 0.928876
[40]	valid_0's binary_logloss: 0.363672	valid_0's auc: 0.933497
[60]	valid_0's binary_logloss: 0.338657	valid_0's auc: 0.936904
[80]	valid_0's binary_logloss: 0.326686	valid_0's auc: 0.939173
[100]	valid_0's binary_logloss: 0.317938	valid_0's auc: 0.941363
[120]	valid_0's binary_logloss: 0.311034	valid_0's auc: 

/opt/anaconda3/envs/FlightRank/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


前10筆機率: [0.99140091 0.64615252 0.43772315 0.01080807 0.00822298 0.63890176
 0.44234942 0.00563728 0.00481942 0.60222049]


/opt/anaconda3/envs/FlightRank/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      0.88      0.93   3678608
           1       0.04      0.88      0.08     21108

    accuracy                           0.88   3699716
   macro avg       0.52      0.88      0.51   3699716
weighted avg       0.99      0.88      0.93   3699716

[[3229070  449538]
 [   2508   18600]]


In [4]:

import os 
# 📌 儲存模型
model_path = os.path.join(save_dir, f"top{n_top}")
os.makedirs(model_path, exist_ok=True)
model_file = os.path.join(model_path, "lgb_classifer_model.txt")

clf.booster_.save_model(model_file)
print(f"✅ 已儲存模型到 'lgb_classifer_model.txt' in {model_file}")


✅ 已儲存模型到 'lgb_classifer_model.txt' in model_output/selected_features_xgb/one_model/top20/lgb_classifer_model.txt


In [5]:
import polars as pl

def compute_topk_hit_rate(df_topk: pl.DataFrame, df_ground_truth: pl.DataFrame, K: int) -> float:
    """
    計算 Top-K 命中率（HitRate@K）
    """
    # 僅保留 Ground Truth 的正例
    df_pos = df_ground_truth.filter(pl.col("selected") == 1).select(["ranker_id"])
    
    # 將每個ranker_id的Top-K取出
    df_topk_k = df_topk.filter(pl.col("rank_in_group") <= K).select(["ranker_id"]).unique()

    # inner join: 有命中的正例
    df_hit = df_pos.join(df_topk_k, on="ranker_id", how="inner")

    num_hit = df_hit.height
    num_total = df_pos.height

    hit_rate = num_hit / num_total if num_total > 0 else 0.0
    
    num_positive = df_ground_truth.filter(pl.col("selected") == 0).height
    print(f"✅ df_val 中 selected=0 的數量: {num_positive}")

    print(f"✅ HitRate@{K}: {hit_rate:.4f} ({num_hit}/{num_total})")
    return hit_rate
# 預測機率
y_val_pred_proba = clf.predict_proba(X_val_np)[:,1]

# 建df_pred
df_pred = df_val.select(["ranker_id"]).with_columns(
    pl.Series("score", y_val_pred_proba)
)

# 排序 rank_in_group
df_pred = df_pred.with_columns(
    pl.col("score").rank("ordinal", descending=True).over("ranker_id").alias("rank_in_group")
)

# 計算Top-K命中率
hit_rate = compute_topk_hit_rate(
    df_topk=df_pred,
    df_ground_truth=df_val,
    K=50
)



/opt/anaconda3/envs/FlightRank/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ df_val 中 selected=0 的數量: 3678608
✅ HitRate@50: 1.0000 (21108/21108)


In [ ]:
num_positive = df_val.filter(pl.col("selected") == 0).height
print(f"✅ df_val 中 selected=0 的數量: {num_positive}")


✅ df_val 中 selected=1 的數量: 3598742


In [12]:
df_pred

ranker_id,score,rank_in_group
str,f64,u32
"""98ce0dabf6964640b63079fbafd42c…",0.994294,1
"""98ce0dabf6964640b63079fbafd42c…",0.679394,2
"""98ce0dabf6964640b63079fbafd42c…",0.526543,6
"""98ce0dabf6964640b63079fbafd42c…",0.004755,16
"""98ce0dabf6964640b63079fbafd42c…",0.006912,14
…,…,…
"""be21a1ace59f41729922828a72c9e3…",0.016521,45
"""be21a1ace59f41729922828a72c9e3…",0.000974,58
"""be21a1ace59f41729922828a72c9e3…",0.015265,47


In [ ]:
# 計算Top-K命中率
hit_rate = compute_topk_hit_rate(
    df_topk=df_pred,
    df_ground_truth=df_val,
    K=
)

✅ df_val 中 selected=0 的數量: 3678608
✅ HitRate@1: 1.0000 (21108/21108)


# Prediction

In [ ]:
import os
import numpy as np
import polars as pl
from lightgbm import Booster

# 參數
model_path = "model_output/selected_features_xgb/one_model/top20/lgb_classifer_model.txt"
parquet_path = "data/test_filled.parquet"

# 讀取模型 (用Booster)
booster = Booster(model_file=model_path)

# 確認 feature_names
model_features = booster.feature_name()
if model_features is None:
    raise ValueError("❌ 模型沒有 feature_names，請確認訓練時有指定 feature_names")
print(f"✅ 模型共 {len(model_features)} 個features")

# 讀取 test_filled
df = pl.read_parquet(parquet_path)
print(f"✅ 讀取 test_filled，共 {df.height} rows")

# 檢查缺失
missing_in_data = [f for f in model_features if f not in df.columns]
if missing_in_data:
    raise ValueError(f"❌ 下列特徵在 test_filled 不存在: {missing_in_data}")

# 篩選
df_for_predict = df.select(model_features)
X_np = df_for_predict.to_numpy()

# 預測 (Booster.predict 直接回傳概率)
preds = booster.predict(X_np)
print(f"✅ 預測完成，共 {len(preds)} 筆")

# 回存結果
df_result = (
    df
    .with_columns([
        pl.Series("selected", preds)
    ])
)


✅ 模型共 20 個features


✅ 讀取 test_filled，共 6897776 rows
✅ 預測完成，共 6897776 筆


In [11]:
import os
import polars as pl

def export_submission_parquets(
    test_filled_with_preds: pl.DataFrame,
    output_dir: str,
    raw_filename: str = "raw_submission.parquet",
    ranked_filename: str = "rank_submission.parquet"
):
    """
    根據 test_filled_with_preds 輸出兩個 parquet:
    1. 原始分數 (selected)
    2. rank 排序 (selected)
    """
    # 檢查目錄
    os.makedirs(output_dir, exist_ok=True)

    # Subset + __index_level_0__
    subset_df = (
        test_filled_with_preds
        .select(["Id", "ranker_id", "selected"])
        .with_columns(
            pl.col("Id").alias("__index_level_0__")
        )
        .with_columns([
            pl.col("Id").cast(pl.Int64),
            pl.col("ranker_id").cast(pl.Utf8),
            pl.col("selected").cast(pl.Float64),
            pl.col("__index_level_0__").cast(pl.Int64)
        ])
    )

    # 儲存原始 parquet
    raw_path = os.path.join(output_dir, raw_filename)
    subset_df.write_parquet(raw_path)
    print(f"✅ 已儲存原始 submission: {raw_path}")
    print(subset_df)

    # Rank 排名
    ranked_df = subset_df.with_columns(
        pl.col("selected")
          .rank(method="ordinal", descending=True)
          .over("ranker_id")
          .alias("selected")
    )

    # 儲存排名 parquet
    ranked_path = os.path.join(output_dir, ranked_filename)
    ranked_df.write_parquet(ranked_path)
    print(f"✅ 已儲存rank submission: {ranked_path}")
    print(ranked_df)
    
n_top = 20
export_submission_parquets(
    test_filled_with_preds=df_result,   # 你的帶有 selected 分數的 DataFrame
    output_dir=f"model_output/selected_features_xgb/one_model/top{n_top}",
    ranked_filename = "lgb_classifer_rank_submission.parquet",
    raw_filename ="lgb_classifer_raw_submission.parquet",
)


✅ 已儲存原始 submission: model_output/selected_features_xgb/one_model/top20/lgb_classifer_raw_submission.parquet
shape: (6_897_776, 4)
┌──────────┬─────────────────────────────────┬──────────┬───────────────────┐
│ Id       ┆ ranker_id                       ┆ selected ┆ __index_level_0__ │
│ ---      ┆ ---                             ┆ ---      ┆ ---               │
│ i64      ┆ str                             ┆ f64      ┆ i64               │
╞══════════╪═════════════════════════════════╪══════════╪═══════════════════╡
│ 18144679 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 0.327041 ┆ 18144679          │
│ 18144680 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 0.664309 ┆ 18144680          │
│ 18144681 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 0.120555 ┆ 18144681          │
│ 18144682 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 0.449987 ┆ 18144682          │
│ 18144683 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 0.309593 ┆ 18144683          │
│ …        ┆ …                               ┆ …        ┆ …                 │
│ 25043143 ┆

In [ ]:
from scripts.result_esemble import compute_ranked_average_selected_from_submissions

submission_paths = [
    "model_output/selected_features_xgb/one_model/top60/raw_submission.parquet",
    "model_output/all_features_xgb/2_raw_submission.parquet",
    "model_output/all_features_xgb/raw_submission.parquet",
    "model_output/selected_features_xgb/one_model/top20/raw_submission.parquet"
]

avg_df = compute_ranked_average_selected_from_submissions(submission_paths)

# 顯示前5筆
print(avg_df.head())

# 如需輸出
avg_df.write_parquet("submission_selected_mean.parquet")
print("✅ 已輸出平均submission")


✅ 讀取: model_output/selected_features_xgb/one_model/top60/raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/2_raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/top20/raw_submission.parquet
✅ 完成 6897776 筆平均+排名計算
shape: (5, 4)
┌──────────┬─────────────────────────────────┬──────────┬───────────────────┐
│ Id       ┆ ranker_id                       ┆ selected ┆ __index_level_0__ │
│ ---      ┆ ---                             ┆ ---      ┆ ---               │
│ i64      ┆ str                             ┆ u32      ┆ i64               │
╞══════════╪═════════════════════════════════╪══════════╪═══════════════════╡
│ 18144679 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 98       ┆ 18144679          │
│ 18144680 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 32       ┆ 18144680          │
│ 18144681 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 223      ┆ 18144681          │
│ 18144682 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 122      ┆ 18